Name: Şirag Erkol

I hereby declare that I observed the honour code of the university when preparing the homework.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math, Latex

## Pr?gr?mm?ng?H?m?w?rk 3

In this exercise we model a string of text using a Markov(1) model. For simplicity we only consider letters 'a-z'. Capital letters 'A-Z' are mapped to the corresponding ones. All remaining letters, symbols, numbers, including spaces, are denoted by '.'.


We have a probability table $T$ where $T_{i,j} = p(x_t = j | x_{t-1} = i)$  transition model of letters in English text for $t=1,2 \dots N$. Assume that the initial letter in a string is always a space denoted as $x_0 = \text{'.'}$. Such a model where the probability table is always the same is sometimes called a stationary model.

1. For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$
1. Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc. Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below. 
1. Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.
1. Discuss how you can improve the model to get better estimations.

In [4]:
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

Hint: The code below loads a table of transition probabilities for English text.

In [5]:
import csv
from IPython.display import display, Latex

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

print('Example')
## p(x_t = 'u' | x_{t-1} = 'q')
display(Latex(r"$p(x_t = \text{'u'} | x_{t-1} = \text{'q'})$"))
print(T[letter2idx['q']][letter2idx['u']])
display(Latex(r"$p(x_t | x_{t-1} = \text{'a'})$"))
for c,p in zip(alphabet,T[letter2idx['a']]):
    print(c,p)

Example


<IPython.core.display.Latex object>

0.9949749


<IPython.core.display.Latex object>

('a', '0.0002835')
('b', '0.0228302')
('c', '0.0369041')
('d', '0.0426290')
('e', '0.0012216')
('f', '0.0075739')
('g', '0.0171385')
('h', '0.0014659')
('i', '0.0372661')
('j', '0.0002353')
('k', '0.0110124')
('l', '0.0778259')
('m', '0.0260757')
('n', '0.2145354')
('o', '0.0005459')
('p', '0.0195213')
('q', '0.0001749')
('r', '0.1104770')
('s', '0.0934290')
('t', '0.1317960')
('u', '0.0098029')
('v', '0.0306574')
('w', '0.0088799')
('x', '0.0009562')
('y', '0.0233701')
('z', '0.0018701')
('.', '0.0715219')


## Part 1

In [6]:
for i in range(len(T)):
    T[i]=list(map(float,T[i]))
for i in range(len(T)):
    T[i]=T[i]/np.sum(T[i])
T=np.vstack(T)

In [7]:
def f1(N):
    text='.'
    for i in range(N):
        letter=np.random.choice(alphabet,1,p=T[alphabet.index(text[i])])
        text=text+letter.tostring()
    print text[1:]

In [8]:
f1(100)

bef.in.bof.manirees.atheser.thas.ce.vie.theele.has.s.ant.mes.hespo.lllte.achitlo.iatred..adeve.as.n.


## Part 2

To find the probability values, we first need to check for the conditional dependencies. Assume we have a string $'a__df_c'$. Let's first number the empty letters: 'a23df6c'. Since we are using a Markov(1) model, it can be said that the empty letters are only dependent on the given last previous letter and the first next letter. In the case of empty cell 2, these letters are 'a' and 'd'. Then, the probability distribution turns out to be as following:
$$p(2|1,4)=\frac{p(1,2,4)}{p(1,4)}=\frac{p(4|2) \times p(2|1) \times p(1)}{p(4|1) \times p(1)}=\frac{p(4|2) \times p(2|1)}{p(4|1)}=\frac{p(4='d'|2) \times p(2|1='a')}{p(4='d'|1='a')}$$<br>
This calculation will give the necessary probability distribution for the second cell, when the cell 1 and 4 are given. The following two parts are done using these calculations. 

In [9]:
def f2(text):
    text='.'+text
    textnew=''
    lastLetter=9999
    if text[len(text)-1]=="_":
        for i in range(len(text)):
            if text[i]!="_":
                lastLetter=i
    for i in range(len(text)):
        if text[i] == '_' :
            matrix1=T
            matrix2=T
            matrix3=T
            if i > lastLetter:
                prob=matrix1[alphabet.index(textnew[i-1])]
            else:
                for j in range(len(text)):
                    if j > i and text[j]!='_':
                        ind2=j
                        break
                if ind2 - i > 1:
                    for k in range(ind2 - i - 1):
                        matrix2=np.dot(matrix2,T)
                for k in range(ind2 - (i - 1) - 1):
                    matrix3=np.dot(matrix3,T)
                prob=np.multiply(matrix1[alphabet.index(textnew[i-1])],matrix2[:,alphabet.index(text[ind2])])/matrix3[alphabet.index(textnew[i-1]),alphabet.index(text[ind2])]
            letter=np.random.choice(alphabet,1,p=prob)
            textnew=textnew+letter.tostring()
        else:
            textnew=textnew+text[i]
    return textnew

In [10]:
for word in test_strings:
    for i in range(10):
        print "Word " + str(test_strings.index(word) + 1) + " - Sample " + str(i) + ": " + f2(word)
    print ""

Word 1 - Sample 0: .the.brlin.fex.
Word 1 - Sample 1: .the.br.an.fex.
Word 1 - Sample 2: .tho.br.in.fex.
Word 1 - Sample 3: .the.br.an.fex.
Word 1 - Sample 4: .thi.br.en.fex.
Word 1 - Sample 5: .thimbr.on.fix.
Word 1 - Sample 6: .theabrern.fex.
Word 1 - Sample 7: .theabr.on.f.x.
Word 1 - Sample 8: .tha.br.in.fex.
Word 1 - Sample 9: .tht.br.on.fex.

Word 2 - Sample 0: .tunstrint.tombe.onsweri.
Word 2 - Sample 1: .ou.stwong.tofbe.answerom
Word 2 - Sample 2: .ounstotny.torbe.onswire.
Word 2 - Sample 3: .mutstsend.to.be.inswhrec
Word 2 - Sample 4: .outsthins.tosbe.answaran
Word 2 - Sample 5: .busstlang.to.be.onswirin
Word 2 - Sample 6: .ourstheng.toobe.answer.d
Word 2 - Sample 7: .ru.st.ond.tombe.answare.
Word 2 - Sample 8: .qurstonnd.to.be.wnswaran
Word 2 - Sample 9: .buasthend.tonbe.answorew

Word 3 - Sample 0: .il.at..gashone.ge.rwing
Word 3 - Sample 1: .it.ats.hathans.fear.ing
Word 3 - Sample 2: .ilmath.cathind.hetrsing
Word 3 - Sample 3: .ighaty.gawhend.begrling
Word 3 - Sample 4: .in

### Part 3

In [11]:
def f3(text):
    text='.'+text
    textnew=''
    lastLetter=9999
    totalProb=1
    if text[len(text)-1]=="_":
        for i in range(len(text)):
            if text[i]!="_":
                lastLetter=i
    for i in range(len(text)):
        if text[i] == '_' :
            matrix1=T
            matrix2=T
            matrix3=T
            if i > lastLetter:
                prob=matrix1[alphabet.index(textnew[i-1])]
            else:
                for j in range(len(text)):
                    if j > i and text[j]!='_':
                        ind2=j
                        break
                if ind2 - i > 1:
                    for k in range(ind2 - i - 1):
                        matrix2=np.dot(matrix2,T)
                for k in range(ind2 - (i - 1) - 1):
                    matrix3=np.dot(matrix3,T)
                prob=np.multiply(matrix1[alphabet.index(textnew[i-1])],matrix2[:,alphabet.index(text[ind2])])/matrix3[alphabet.index(textnew[i-1]),alphabet.index(text[ind2])]
            totalProb=totalProb*max(prob)
            letter=alphabet[int(np.where(prob==max(prob))[0])]
            textnew=textnew+letter
        else:
            textnew=textnew+text[i]
    return textnew, totalProb

In [12]:
for word in test_strings:
    print "Word " + str(test_strings.index(word) + 1) + ": " + str(f3(word)[0])
    print "log-probability:", np.log(f3(word)[1])
    print ""

Word 1: .the.br.an.fex.
log-probability: -3.07433488138

Word 2: .oursthend.to.be.answere.
log-probability: -10.8154510862

Word 3: .in.ath.wathend.he.r.ing
log-probability: -11.6360900332

Word 4: .qur.t.thiz.the.at.an.the.an.
log-probability: -22.9236427638



### Part 4

To improve the model for a better performance, we can use Markov(i) where i is greater than 1, such as Markov(4). This is because the probability of letters do not depend only on the previous letter, they can also depend on the ones before those or the ones coming after itself. So, a higher Markov model would give better performances, although the complexity would also be higher.